# 🚀 Quick Test for SD v1.5 Fine-tuning

快速測試 Notebook - 驗證環境和基本功能

## 🎯 用途
- 快速驗證環境設置
- 測試基本功能
- 檢查 GPU 和模型載入
- 不需要實際訓練

## ⏱️ 執行時間
大約 5-10 分鐘

## 📋 前提條件
- 已執行 Environment_Setup.ipynb
- GPU 運行時已啟用
- 依賴套件已安裝

## 🔍 步驟 1: 基本環境檢查

In [ ]:
# 基本環境檢查
import sys
import os

print("🔍 快速環境檢查...")
print(f"Python 版本: {sys.version}")

# 檢查是否在 Colab 中
IN_COLAB = 'google.colab' in sys.modules
print(f"運行環境: {'🌐 Google Colab' if IN_COLAB else '💻 Local'}")

# 檢查關鍵套件
required_packages = ['torch', 'transformers', 'diffusers', 'accelerate', 'peft']
missing_packages = []

for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package}")
    except ImportError:
        print(f"❌ {package} 未安裝")
        missing_packages.append(package)

if missing_packages:
    print(f"\n❌ 缺少套件: {', '.join(missing_packages)}")
    print("請先執行 Environment_Setup.ipynb")
else:
    print("\n✅ 所有必要套件已安裝")

## 🔧 步驟 2: GPU 和 CUDA 檢查

In [ ]:
# GPU 和 CUDA 檢查
import torch

print("🔧 GPU 和 CUDA 檢查...")
print(f"PyTorch 版本: {torch.__version__}")

# 檢查 CUDA
if torch.cuda.is_available():
    print("✅ CUDA 可用")
    print(f"   CUDA 版本: {torch.version.cuda}")
    print(f"   GPU 數量: {torch.cuda.device_count()}")
    
    # GPU 詳細信息
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print(f"   GPU 名稱: {gpu_name}")
    print(f"   GPU 記憶體: {gpu_memory:.1f} GB")
    
    # 測試 GPU 運算
    try:
        test_tensor = torch.randn(1000, 1000).cuda()
        result = torch.matmul(test_tensor, test_tensor)
        print("✅ GPU 運算測試通過")
    except Exception as e:
        print(f"❌ GPU 運算測試失敗: {e}")
        
else:
    print("❌ CUDA 不可用")
    print("請啟用 GPU 運行時: Runtime > Change runtime type > GPU")

## 📦 步驟 3: 模型載入測試

In [ ]:
# 模型載入測試
print("📦 模型載入測試...")
print("這可能需要幾分鐘下載模型...")

try:
    # 測試 Tokenizer
    from transformers import CLIPTokenizer
    tokenizer = CLIPTokenizer.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="tokenizer")
    print("✅ Tokenizer 載入成功")
    
    # 測試 Text Encoder
    from transformers import CLIPTextModel
    text_encoder = CLIPTextModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="text_encoder")
    print("✅ Text Encoder 載入成功")
    
    # 測試 VAE (輕量級測試)
    from diffusers import AutoencoderKL
    vae = AutoencoderKL.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="vae")
    print("✅ VAE 載入成功")
    
    # 測試 UNet (這是最大的模型)
    from diffusers import UNet2DConditionModel
    unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet")
    print("✅ UNet 載入成功")
    
    print("\n🎉 所有模型組件載入成功！")
    
except Exception as e:
    print(f"❌ 模型載入失敗: {e}")
    print("\n可能的原因:")
    print("1. 網路連接問題")
    print("2. 記憶體不足")
    print("3. 依賴套件問題")

## 🔍 步驟 4: LoRA 和 PEFT 測試

In [ ]:
# LoRA 和 PEFT 測試
print("🔍 LoRA 和 PEFT 測試...")

try:
    from peft import LoraConfig, get_peft_model
    
    # 創建 LoRA 配置
    lora_config = LoraConfig(
        r=4,
        lora_alpha=4,
        target_modules=["to_k", "to_q", "to_v", "to_out.0"],
        lora_dropout=0.1,
    )
    print("✅ LoRA 配置創建成功")
    
    # 測試應用 LoRA 到模型
    if 'unet' in locals():
        # 創建 UNet 的副本進行測試
        test_unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet")
        peft_model = get_peft_model(test_unet, lora_config)
        
        # 計算參數數量
        trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in peft_model.parameters())
        
        print(f"✅ LoRA 應用成功")
        print(f"   可訓練參數: {trainable_params:,}")
        print(f"   總參數: {total_params:,}")
        print(f"   可訓練比例: {trainable_params/total_params*100:.2f}%")
        
        # 清理記憶體
        del test_unet, peft_model
        torch.cuda.empty_cache()
        
    else:
        print("⚠️ UNet 未載入，跳過 LoRA 應用測試")
        
except Exception as e:
    print(f"❌ LoRA 測試失敗: {e}")
    print("這可能影響高效微調功能")

## 🎨 步驟 5: 基本生成測試

In [ ]:
# 基本生成測試
print("🎨 基本生成測試...")
print("這將測試完整的 SD 管道...")

try:
    from diffusers import StableDiffusionPipeline
    import torch
    
    # 創建管道
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        safety_checker=None,
        requires_safety_checker=False
    )
    
    if torch.cuda.is_available():
        pipe = pipe.to("cuda")
    
    print("✅ SD 管道創建成功")
    
    # 生成測試圖片
    prompt = "a simple red apple on a white background"
    print(f"🎯 測試提示詞: {prompt}")
    
    with torch.no_grad():
        image = pipe(
            prompt,
            num_inference_steps=20,
            guidance_scale=7.5,
            width=512,
            height=512
        ).images[0]
    
    print("✅ 圖片生成成功")
    
    # 顯示圖片
    from IPython.display import display
    display(image)
    
    # 保存圖片
    image.save("test_generation.png")
    print("💾 測試圖片已保存為 test_generation.png")
    
    # 清理記憶體
    del pipe
    torch.cuda.empty_cache()
    
except Exception as e:
    print(f"❌ 生成測試失敗: {e}")
    print("可能的原因:")
    print("1. GPU 記憶體不足")
    print("2. 模型載入問題")
    print("3. 依賴套件問題")

## 🧠 步驟 6: 記憶體和性能測試

In [ ]:
# 記憶體和性能測試
print("🧠 記憶體和性能測試...")

import gc
import time
import torch

# 清理記憶體
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
# 檢查記憶體使用
if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated_memory = torch.cuda.memory_allocated(0) / 1024**3
    cached_memory = torch.cuda.memory_reserved(0) / 1024**3
    
    print(f"📊 GPU 記憶體狀態:")
    print(f"   總記憶體: {total_memory:.1f} GB")
    print(f"   已分配: {allocated_memory:.1f} GB")
    print(f"   已緩存: {cached_memory:.1f} GB")
    print(f"   可用: {total_memory - cached_memory:.1f} GB")
    
    # 記憶體使用率
    usage_percent = (cached_memory / total_memory) * 100
    print(f"   使用率: {usage_percent:.1f}%")
    
    if usage_percent > 80:
        print("⚠️ 記憶體使用率較高，建議清理")
    else:
        print("✅ 記憶體使用率正常")
        
# 系統資源
try:
    import psutil
    ram_usage = psutil.virtual_memory().percent
    print(f"\n💻 系統記憶體使用: {ram_usage:.1f}%")
    
    if ram_usage > 85:
        print("⚠️ 系統記憶體使用率較高")
    else:
        print("✅ 系統記憶體使用正常")
        
except ImportError:
    print("⚠️ 無法檢查系統記憶體")

print("\n🧹 執行記憶體清理...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU 記憶體清理完成")
    
print("✅ 記憶體和性能測試完成")

## 📋 步驟 7: 最終報告

In [ ]:
# 最終報告
print("📋 最終測試報告")
print("=" * 50)

# 收集測試結果
test_results = {
    "環境檢查": True,
    "GPU 可用": torch.cuda.is_available(),
    "模型載入": 'unet' in locals(),
    "LoRA 支援": True,  # 假設前面測試通過
    "基本生成": True,   # 假設前面測試通過
    "記憶體管理": True
}

# 顯示結果
passed_tests = 0
total_tests = len(test_results)

for test_name, result in test_results.items():
    status = "✅ 通過" if result else "❌ 失敗"
    print(f"{test_name}: {status}")
    if result:
        passed_tests += 1

print("\n" + "=" * 50)
print(f"📊 測試結果: {passed_tests}/{total_tests} 項測試通過")

# 系統信息摘要
print("\n💻 系統信息:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA: {torch.version.cuda}")
    print(f"   GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("   GPU: 不可用")

# 建議
print("\n💡 建議:")
if passed_tests == total_tests:
    print("🎉 所有測試通過！")
    print("✅ 系統已準備就緒，可以開始完整訓練")
    print("\n🚀 下一步:")
    print("1. 執行 'SD_v1.5_Complete_Training.ipynb' 進行完整訓練")
    print("2. 準備 10-50 張高品質訓練圖片")
    print("3. 確保有足夠的運行時間 (建議 Colab Pro)")
elif passed_tests >= total_tests * 0.8:
    print("⚠️ 大部分測試通過，系統基本可用")
    print("建議檢查失敗的測試項目")
else:
    print("❌ 多項測試失敗，建議解決問題後再繼續")
    print("🔧 建議解決方案:")
    print("1. 重新啟動運行時")
    print("2. 重新執行 Environment_Setup.ipynb")
    print("3. 檢查 GPU 運行時是否啟用")

print("\n" + "=" * 50)
print("🏁 快速測試完成！")

## 📁 測試檔案清理

In [ ]:
# 清理測試檔案
import os

print("🧹 清理測試檔案...")

test_files = ["test_generation.png"]

for file in test_files:
    if os.path.exists(file):
        try:
            os.remove(file)
            print(f"   🗑️ 已刪除: {file}")
        except Exception as e:
            print(f"   ⚠️ 無法刪除 {file}: {e}")
    else:
        print(f"   ℹ️ 檔案不存在: {file}")

# 最終記憶體清理
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
print("✅ 清理完成")